In [1]:
import yaml
import evaluate
import json

def em(row):
    ind = row["output"].find(row["q"])
    mo = row["output"][ind:].strip("</s>")
    mo = mo.strip("<|endoftext|>")
    mo = post_process(mo)
#     print("mo : \n", mo)
    gt = row["q"] + "\n" + row["o"]
#     print("gt: \n", gt)
    return mo == gt

bleu = evaluate.load("bleu")
def blu(row):
    ind = row["output"].find(row["q"])
    mo = row["output"][ind:].strip("</s>")
    mo = mo.strip("<|endoftext|>")
    mo = post_process(mo)
#     print("mo : \n", mo)
    gt = row["q"] + "\n" + row["o"]
#     print("gt: \n", gt)
    return bleu.compute(predictions=[mo], references=[gt])["bleu"]
    
    
import jsonschema
import pandas as pd

df = pd.read_parquet("./module_to_schema_64_missing.parquet")
df = df.to_dict(orient="records")
getschema = {}
for d in df:
    getschema[d["module"]] = d["schema"]

def schemac(row):
    try:
        schema = json.loads(getschema[row["module"]])
    except Exception as e:
        print(e)
        return False
    ind = row["output"].find(row["q"])
    mo = row["output"][ind:].strip("</s>")
    mo = mo.strip("<|endoftext|>")
    mo = post_process(mo)
    try:
        mo = yaml.safe_load(mo)
        if isinstance(mo, list):
            mo = mo[0]
        del mo["name"]
#         print(mo)
    except Exception as e:
        print(e)
        return False
    try:
        jsonschema.validate(mo, schema)
    except Exception as e:
        print(e)
        return False 
    return True

# def ansible_aware(row):
#     ind = row["output"].find(row["q"])
#     mo = row["output"][ind:].strip("</s>")
#     gt = row["q"] + "\n" + row["o"]
#     try:
#         mo = yaml.safe_load(mo)
#         gt = yaml.safe_load(gt)
#     except Exception as e:
        
    
def post_process(yamlt):
    ymltlines = yamlt.split("\n")
    for i in range(len(ymltlines)):
        ymltlines[i] = ymltlines[i].lstrip("#")
    return "\n".join(ymltlines)


/raid/nlp/sameer/anaconda3/envs/guidance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'jsonschema'

In [53]:
!pip install parsimonious

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import guidance
import json
import transformers
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

/raid/nlp/sameer/anaconda3/envs/guidance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
class nl2structure:
    def __init__(self, model: str, prompt, task="prompting", schema=None, reference_module=None, template=None, token_healing=True):
        self.schema = schema
        self.reference_module = reference_module
        self.tokenizer = AutoTokenizer.from_pretrained(model)
#         self.model = AutoModelForCausalLM.from_pretrained(model, ignore_mismatched_sizes=True).to("cuda")
        self.task = task
        self.template = template
        self.prompt = prompt
        self.token_healing = token_healing
        self.model = guidance.llms.Transformers(model)
#         guidance.llm = guidance.llms.transformers.MPTChat(model=model,tokenizer=self.tokenizer)


    def return_required_keys(self, contents):
            
        dic = contents[self.reference_module]
        required_keys = []
        key_schema = {}
        for key in dic["doc"]["options"]:
          if "required" in dic["doc"]["options"][key].keys():
            if dic["doc"]["options"][key]["required"]==True:
              required_keys.append(key)
                
        for key in required_keys:
          key_schema[key] = []
          if "suboptions" in dic["doc"]["options"][key]:
            for key_1 in dic["doc"]["options"][key]["suboptions"]:
              if "required" in dic["doc"]["options"][key]["suboptions"][key_1] and dic["doc"]["options"][key]["suboptions"][key_1]["required"]==True:
                key_schema[key].append(key_1)
                
        return(key_schema) 
        
    def return_template_elements(self):
        
        with open('/raid/nlp/sameer/guidance_updated/guidance/guidance-guidanceV2/data_with_ft.jsonl', 'r') as json_file:
          json_list = list(json_file)

        contents = {}
        for json_str in json_list:
          result = json.loads(json_str)
          if result['ansible_module'] in result:
            contents[result['ansible_module']] = result[result['ansible_module']]

        required_keys = self.return_required_keys(contents)

        all_keys = list(contents[self.reference_module]["doc"]["options"].keys())
        optional_keys = []
        for i in all_keys:
          if i not in required_keys:
            optional_keys.append(i)
        return required_keys, optional_keys
    
    def build_template(self):
      if self.task == "ansible_yaml":

        # template for ansible is built as all the level 1 keys in select and select in geneach with max_iteration = no. of keys there is gen after every select for value generation 
        required_keys, optional_keys = self.return_template_elements()
        max_iter = len(required_keys.keys()) + len(optional_keys)
        guidance_template = """{{#geneach "items" min_iterations=0 max_iterations=""" + str(max_iter) +  """ unique=unique}}{{#select "key" id=id}}"""
        for key in list(required_keys.keys()) + optional_keys[:-1]:
          guidance_template += key + "{{or}}"

        guidance_template += optional_keys[-1] + "{{/select}}" + """{{gen "value" dic=options_available}}{{/geneach}}"""
        self.template = guidance_template
    
    def __call__(self): 
      
      if self.template:
        final_prompt = self.prompt
      else:
        
        if self.schema and self.reference_module:

          
          if self.task == "ansible_yaml":
            module_line = self.prompt.split("\n")[-1]
            module_selection_prompt  = self.prompt + "\n" + " "*cnt_spaces_left(module_line) + "  " + """{{select "module" options=module_list cmd_name=True}}"""
            print(module_selection_prompt)
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            self.reference_module = eval(self.reference_module)
            prog_module = guidance(module_selection_prompt, token_healing=self.token_healing, llm=self.model)
            module_opt = prog_module(module_list=self.reference_module)
            print(str(module_opt))

            self.reference_module = guidance.library._select.selected_module
            self.build_template()
            final_prompt = self.prompt + "\n" + " "*cnt_spaces_left(module_line) + "  "  + self.reference_module + "\n" + " "*cnt_spaces_left(module_line) + "  " + self.template
            regex_pattern = ".*\n\s{" + str(cnt_spaces_left(module_line) + 2) + "}$"    #regex pattern and it's corresponding template to change.
                                                                                        #for ansible we need regex pattern for level 1 keys. Which is 2 more spaces than module name line
            options_available = {regex_pattern: self.template}

      # options_available = None
      task_details = {"schema_path": self.schema, "task": self.task, "module_name": self.reference_module, "options_available": options_available}

#       print(final_prompt)
#       print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
#       print(options_available)
      prog1 = guidance(final_prompt, token_healing=self.token_healing, module=self.reference_module, options=options_available, schema=self.schema, task_config=task_details, llm=self.model)
      out = prog1(options_available=options_available, unique=[1], id=1)
      print(str(out))


In [3]:
def cnt_spaces_left(text):
  return (len(text)-len(text.lstrip(' ')))


In [14]:
i = 25

In [57]:
print(repr(tokenizer.decode([29871])))
print(repr(tokenizer.decode([1678])))
print(repr(tokenizer.decode([3909, 44, 32229, 81, 10419])))
print(tokenizer.encode(' register: nat_gateway'))

'efa'
' https'
' register: nat_gateway'
[3909, 44, 32229, 81, 10419]


In [4]:
print(''' - name: Delete the application security group\n  azure.azcollection.azure_rm_applicationsecuritygroup:\n    state: absent\n   {{#geneach "items" min_iterations=0 max_iterations=2 unique=unique}}{{#select "key" id=id}}  name{{or}}  resource_group{{/select}}:{{gen "value"}}{{/geneach}}''')

 - name: Delete the application security group
  azure.azcollection.azure_rm_applicationsecuritygroup:
    state: absent
     resource_group


In [8]:
# print('''    - name: cleanup topics\n      community.general.onepassword_info:\n        search_terms:\n          - My OnePassword\n          - My Other OnePassword\n       vault''')
# print('''    - name: cleanup topics\n      community.general.onepassword_info:\n        search_terms:\n          - My OnePassword\n          - My Other OnePassword\n        auto_login:\n         {{#geneach "items" min_iterations=0 max_iterations=4 unique=unique}}{{#select "key" id=id}} master_password{{or}} secret_key{{or}} subdomain{{or}} username{{/select}}:{{gen "value" dic=options_available}}{{/geneach}}''')
print('''    - name: cleanup topics\n      community.general.onepassword_info:\n        search_terms:\n          - My OnePassword\n          - My Other OnePassword\n        auto_login:\n          master_password::{{gen "value"}}''')
# print(repr('    '))
# print(repr('     '))

    - name: cleanup topics
      community.general.onepassword_info:
        search_terms:
          - My OnePassword
          - My Other OnePassword
        auto_login:
          master_password::{{gen "value"}}


In [ ]:
[' library_id', ' vcenter_username', '  vcenter_hostname', ' vcenter_password']
[' vcenter_hostname', ' vcenter_password', ' vcenter_username', ' library_id', ' session_timeout', ' vcenter_rest_log_file', ' vcenter_validate_certs']

In [ ]:
' vcenter_hostname'
'  vcenter_hostname'

In [ ]:
'  '

In [ ]:
tokenizer.decode([31,   636,    44,  2648,  6687,  7663,  2350,   225,    35,   334])
tokenizer.decode([31,   636,    44,  2648,  6687,  7663,  2350,   225,    35,   334])

In [18]:
print(tokenizer.decode([81, 965, 44]))
print(repr(tokenizer.decode([3335])))

_url:
' {{'


In [2]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-1b")

print(tokenizer.encode('''    - name: cleanup topics
      community.general.onepassword_info:
        search_terms:
          - My OnePassword
          - My Other OnePassword
        auto_login:
          master_password: MySuperSecureP4sswOrd
          auto_login_timeout: 10000
      register: resul'''))

print(tokenizer.encode('''name: Show openstack subnets
  openstack.cloud.subnets_info:
    auth:
    {{gen "value"}}'''))

NameError: name 'AutoTokenizer' is not defined

In [15]:

ir_data_df = pd.read_parquet("/raid/nlp/sameer/guidance-guidanceV2/random_85_15_split_test_ir_inf.parquet")
module_list = ir_data_df['top_3_colbert_ft'][i]

prompt = ir_data_df['c'][i] + "\n" + ir_data_df['q'][i]
prompt = prompt.replace("{", "\\{")

obj = nl2structure(schema = "/raid/nlp/sameer/guidance_updated/guidance/guidance-guidanceV2/data_with_ft.jsonl", reference_module = module_list, model="bigcode/starcoderbase-1b", prompt=prompt, task="ansible_yaml", template=False)
pred_structure = obj()
print(pred_structure)

Stop program # Copyright 2019 Google Inc.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

---
# ----------------------------------------------------------------------------
#
# *** AUTO GENERATED CODE *** AUTO GENERATED CODE ***
#
# ----------------------------------------------------------------------------
#
# This file is automatically generated by Magic Modules and manual
# changes will be clobbered when the file is regenerated.
#
# Please read more about how to change this file at
# https://www.github.com/GoogleCloudPlatform/magic-modules
#
# ----------------------------------------------------------------------------
# Pre-test setup
- name: delete a firewall
 google.cloud.gcp_compute_firewall:
 name: "\{\{ resource_name }}"
 allowed:
 - ip_protocol: tcp
 ports:
 - '22'
 target_tags:
 - test-ssh-server
 - staging-ssh-server
 source_tags:
 - test-ssh-clients
 project: "\{\{ gcp_project }}"
 auth_kind: "\{\{ gcp_cred_kind }}"
 service_account_file: "\{\{ gcp_cred_file }}"
 state: absent
#----------------------------------------------------------
- name: create a firewall
 google.cloud.gcp_compute_firewall:
 name: "\{\{ resource_name }}"
 allowed:
 - ip_protocol: tcp
 ports:
 - '22'
 target_tags:
 - test-ssh-server
 - staging-ssh-server
 source_tags:
 - test-ssh-clients
 project: "\{\{ gcp_project }}"
 auth_kind: "\{\{ gcp_cred_kind }}"
 service_account_file: "\{\{ gcp_cred_file }}"
 state: present
 register: result
- name: assert changed is true
 assert:
 that:
 - result.changed == true
- name: verify that firewall was created
 google.cloud.gcp_compute_firewall_info:
 filters:
 - name = \{\{ resource_name }}
 project: "\{\{ gcp_project }}"
 auth_kind: "\{\{ gcp_cred_kind }}"
 service_account_file: "\{\{ gcp_cred_file }}"
 scopes:
 - https://www.googleapis.com/auth/compute
 register: results
- name: verify that command succeeded
 assert:
 that:
 - results['resources'] | length == 1
# ----------------------------------------------------------------------------
- name: update the firewall
 google.cloud.gcp_compute_firewall:
 name: "\{\{ resource_name }}"
 allowed:
 - ip_protocol: tcp
 ports:
 - '55'
 target_tags:
 - test-ssh-server
 - staging-ssh-server
 source_tags:
 - test-ssh-clients
 project: "\{\{ gcp_project }}"
 auth_kind: "\{\{ gcp_cred_kind }}"
 service_account_file: "\{\{ gcp_cred_file }}"
 state: present
 register: result
- name: assert changed is true
 assert:
 that:
 - result.changed == true
# ----------------------------------------------------------------------------
- name: check firewall was updated.
 community.general.ufw
 <span style='opacity: 1.0; display: inline; background-color: rgba(165, 165, 165, 0.1);' title='{{#geneach "items" min_iterations=0 max_iterations=20 unique=unique}}{{#select "key" id=id}}comment{{or}}default{{or}}delete{{or}}direction{{or}}from_ip{{or}}from_port{{or}}insert{{or}}insert_relative_to{{or}}interface{{or}}interface_in{{or}}interface_out{{or}}log{{or}}logging{{or}}name{{or}}proto{{or}}route{{or}}rule{{or}}state{{or}}to_ip{{or}}to_port{{/select}}{{gen "value" dic=options_available}}{{/geneach}}'> state : present
 register: ufw_result
- {{gen "value" dic=options_available}} {{#geneach "items" min_iterations=0 max_iterations=20 unique=unique}} {{#select "key" id=id}}comment {{or}} default {{or}} delete {{or}} direction {{or}} from_ip {{or}} from_port {{or}} insert {{or}} insert_relative_to {{or}} interface {{or}} interface_in {{or}} interface_out {{or}} log {{or}} logging {{or}} name {{or}} proto 

# Copyright 2019 Google Inc.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

---
# ----------------------------------------------------------------------------
#
#     ***     AUTO GENERATED CODE    ***    AUTO GENERATED CODE     ***
#
# ----------------------------------------------------------------------------
#
#     This file is automatically generated by Magic Modules and manual
#     changes will be clobbered when the file is regenerated.
#
#     Please read more about how to change this fil

In [ ]:
starcode_output_ft.columns

In [27]:
def mmatch(mp, module, q):
    ind = mp.rfind(q)
#     print(module)
    try:
        mpm = mp[ind:].split("\n")[1]
    except Exception:
        return 0
#     print(mpm)
    module = module.split(".")[-1]
    if module in mpm:
        return 1
    failed_samples.append((module, mpm))
    return 0

In [127]:
failed_samples = []
mscore = 0
emscore = 0
score = 0
sscore = 0
bscore = 0
# starcode_output_ft = pd.read_parquet("./tldr_type_split/starcoder_1b_ft_e2_tldr.parquet")
starcode_output_ft = pd.read_parquet("./random_slit_85_15/codellama_7b_base_rand_noc.parquet")
print(starcode_output_ft.columns)

for i in range(len(starcode_output_ft)):
    starcode_output_ft_out = starcode_output_ft["output"][i]
    gt = starcode_output_ft["o"][i]
    mod = starcode_output_ft["module"][i]
    mscore += mmatch(starcode_output_ft_out, mod, starcode_output_ft["q"][i])
    emscore += em(starcode_output_ft.iloc[i])
    bscore += blu(starcode_output_ft.iloc[i])
    sscore += schemac(starcode_output_ft.iloc[i])

Index(['module', 'q', 'c', 'o', 'ground_truth_output_token_count', 'output',
       'output_spliced'],
      dtype='object')
'name' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['name']}

On instance:
    {}
'name' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['name']}

On instance:
    {}
expected a single document in the stream
  in "<unicode string>", line 1, column 1:
    ame: Install a specific version  ... 
    ^
but found another document
  in "<unicode string>", line 4, column 1:
    ---
    ^
while scanning a simple key
  in "<unicode string>", line 6, column 7:
          es_xpack_
          ^
could not find expected ':'
  in "<unicode string>", line 6, column 16:
          es_xpack_
                   ^
while scanning a simple key
  in "<unicode string>", line 6, column 7:
          es_xpack_
          ^
could not find expected ':'
  in "<unicode string>", line 6, column 16:
      

{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
'name'
mapping values are not allowed here
  in "<unicode string>", line 2, column 14:
      description: Removes a cloud AP
                 ^
{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [

{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
while parsing a block collection
  in "<unicode string>", line 1, column 1:
    - name: Remove interface policy  ... 
    ^
expected <block end>, but found '?'
  in "<unicode string>", line 3, column 1:
    tags:
    ^
'datacenter' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['datacenter']}

On instance:
    {}
'api_token' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['api_token']}

On instance:
    {}
{} is not valid under any of the given schemas

Failed validating 'anyOf' in schema['allOf'][0]:
    {'anyOf': [{'required': ['name']}, {'required': ['pattern']}]}

On instance:
    {}
while scanning a quoted scalar
  in "<unicode string>", line 3, column 9:
    - name: "
            ^
found unexpe

while scanning a quoted scalar
  in "<unicode string>", line 6, column 11:
      - name: "Remove Idemp
              ^
found unexpected end of stream
  in "<unicode string>", line 6, column 24:
      - name: "Remove Idemp
                           ^
{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['hostname']}, {'required': ['host']}]}

On instance:
    {}
'switch_type' is a required property

Failed validating 'required' in schema['allOf'][0]['allOf'][1]:
    {'required': ['switch_type']}

On instance:
    {}
while scanning a simple key
  in "<unicode string>", line 45, column 1:
    puppet::master::puppet_master_ca ... 
    ^
could not find expected ':'
  in "<unicode string>", line 45, column 43:
     ... ster::puppet_master_ca_allow_ms_
                                         ^
while scanning a simple key
  in "<unicode string>", line 5, column 1:
    This example shows how to run pu ... 
    ^
could not

{} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['allOf'][0]:
    {'oneOf': [{'required': ['name']}, {'required': ['vs']}]}

On instance:
    {}
expected a single document in the stream
  in "<unicode string>", line 1, column 1:
    - name: Set motd
    ^
but found another document
  in "<unicode string>", line 9, column 1:
    ---
    ^
'name' is a required property

Failed validating 'required' in schema['allOf'][0]['allOf'][0]:
    {'required': ['name']}

On instance:
    {}
while scanning for the next token
found character '`' that cannot start any token
  in "<unicode string>", line 6, column 1:
    ```
    ^
'datacenter_name' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['datacenter_name']}

On instance:
    {}
'name' is a required property

Failed validating 'required' in schema['allOf'][0]['allOf'][0]:
    {'required': ['name']}

On instance:
    {}
{} is not valid under any of the given schemas


...

while scanning a simple key
  in "<unicode string>", line 2, column 1:
    func (s *StoragePoolService) Get ... 
    ^
could not find expected ':'
  in "<unicode string>", line 3, column 1:
    	path := storagePoolPath(name)
    ^
'name'
'name'
'name'
while scanning a simple key
  in "<unicode string>", line 20, column 11:
              descripti
              ^
could not find expected ':'
  in "<unicode string>", line 20, column 20:
              descripti
                       ^
while scanning a simple key
  in "<unicode string>", line 27, column 1:
    control
    ^
could not find expected ':'
  in "<unicode string>", line 27, column 8:
    control
           ^
while constructing a mapping
  in "<unicode string>", line 11, column 17:
              name: {{ .Params.name }}
                    ^
found unhashable key
  in "<unicode string>", line 11, column 18:
              name: {{ .Params.name }}
                     ^
'name'
mapping values are not allowed here
  in "<unicode strin

while scanning a simple key
  in "<unicode string>", line 2, column 1:
    kustomize build --load_restricto ... 
    ^
could not find expected ':'
  in "<unicode string>", line 4, column 1:
    name: Render MutatingWebhooksCon ... 
    ^
while scanning a simple key
  in "<unicode string>", line 16, column 9:
            us
            ^
could not find expected ':'
  in "<unicode string>", line 16, column 11:
            us
              ^
'name' is a required property

Failed validating 'required' in schema['allOf'][0]:
    {'required': ['name']}

On instance:
    {'ame': 'Drain node "foo", but abort if there are pods not managed by '
            'a ReplicationController, ReplicaSet, or Job\n'
            'kubectl drain foo --ignore-daemonsets'}
'name'
'name'
while constructing a mapping
  in "<unicode string>", line 7, column 16:
      minReplicas: {{ .Values.minReplicas }}
                   ^
found unhashable key
  in "<unicode string>", line 7, column 17:
      minReplicas: {{ .Val

mapping values are not allowed here
  in "<unicode string>", line 3, column 15:
     **IMPORTANT**: Enable Data Guard on Autonomou ... 
                  ^
'name'
mapping values are not allowed here
  in "<unicode string>", line 2, column 26:
      oci_autonomous_database:
                             ^
mapping values are not allowed here
  in "<unicode string>", line 2, column 11:
      resource: Oracle::Database
              ^
mapping values are not allowed here
  in "<unicode string>", line 2, column 26:
      oci_autonomous_database:
                             ^
while scanning a simple key
  in "<unicode string>", line 5, column 1:
    Updates the properties of an Aut ... 
    ^
could not find expected ':'
  in "<unicode string>", line 9, column 1:
    - An [Autonomous Transaction Pro ... 
    ^
'name'
mapping values are not allowed here
  in "<unicode string>", line 2, column 26:
      oci_autonomous_database:
                             ^
while scanning a simple key
  in "<un

'name'
while scanning a simple key
  in "<unicode string>", line 4, column 1:
    This operation changes the compa ... 
    ^
could not find expected ':'
  in "<unicode string>", line 9, column 1:
    The following action is required ... 
    ^
while scanning for the next token
found character '`' that cannot start any token
  in "<unicode string>", line 39, column 1:
    ```
    ^
mapping values are not allowed here
  in "<unicode string>", line 2, column 5:
      on:
        ^
'name'
while scanning a simple key
  in "<unicode string>", line 5, column 1:
    Delete a capture filter.
    ^
could not find expected ':'
  in "<unicode string>", line 7, column 1:
    **URL**
    ^
mapping values are not allowed here
  in "<unicode string>", line 2, column 14:
      description: Delete a cpe
                 ^
'name'
while scanning a simple key
  in "<unicode string>", line 5, column 1:
    This operation deletes a cross-c ... 
    ^
could not find expected ':'
  in "<unicode string>", line

'name'
while scanning a simple key
  in "<unicode string>", line 7, column 1:
    The NTP configuration is adjuste ... 
    ^
could not find expected ':'
  in "<unicode string>", line 11, column 1:
    
    ^
'name'
'name'
mapping values are not allowed here
  in "<unicode string>", line 2, column 5:
      on:
        ^
while scanning a quoted scalar
  in "<unicode string>", line 23, column 20:
              license: 'MIT
                       ^
found unexpected end of stream
  in "<unicode string>", line 23, column 24:
              license: 'MIT
                           ^
'name'
mapping values are not allowed here
  in "<unicode string>", line 2, column 14:
      description: List Local Content Library
                 ^
while scanning a quoted scalar
  in "<unicode string>", line 12, column 5:
      - "python >= 2.6
        ^
found unexpected end of stream
  in "<unicode string>", line 12, column 19:
      - "python >= 2.6
                      ^
'name'
mapping values are not all

In [128]:
mscore / len(starcode_output_ft)

0.016662862360191738

In [129]:
emscore / len(starcode_output_ft)

0.0

In [130]:
bscore / len(starcode_output_ft)

0.21033218286924144

In [131]:
sscore / len(starcode_output_ft)

0.09153161378680666

In [54]:
print(failed_samples)

[('aci_node_mgmt_epg', '#  command: "{{ ansible_playbook_dir }}/remove_node_mgmt_out_of_band_epg.yml"'), ('aci_node_mgmt_epg', '#  command: "{{ ansible_playbook_dir }}/remove_node_mgmt_in_band_epg.yml"'), ('aci_node_mgmt_epg', '#      cisco.iosxr.iosxr_ospf_epg:'), ('aci_node_mgmt_epg', 'username_0: <!--\r'), ('aci_node_mgmt_epg', '#  command: "{{ playbook_dir }}/add_node_mgmt_out_of_band_epg_in_check_mode.yml"'), ('aci_node_mgmt_epg', '#      cisco.iosxr.iosxr_eigrp_epg:'), ('aci_node_mgmt_epg', 'username_0: <!--\r'), ('aci_node_mgmt_epg', '#  - name: Add'), ('gcp_compute_firewall_info', '#   shell'), ('gcp_compute_firewall_info', ''), ('gcp_compute_firewall_info', ''), ('gcp_compute_firewall_info', ''), ('gcp_compute_firewall_info', ''), ('gcp_compute_firewall_info', '#   command:'), ('gcp_compute_firewall_info', '#   shell'), ('gcp_compute_firewall_info', '#   shell: |'), ('vmware_host_ntp_info', '#      hosts: localhost'), ('vmware_host_ntp_info', '#      hosts: localhost'), ('vmwa

In [66]:
pred_structure = obj()
print(pred_structure)

Stop program 
name: Create an SNMP user
 community.network.ce_snmp_user
 auth_protocol : MD5
 auth_passphrase: secret
 auth_user: admin
 auth_host: 10.10.10.10
 auth_port: 389
 auth_context: user
 auth_passphrase_hash: true
 auth_passphrase_algorithm: sha
 auth_passphrase_salt: 123456789012345678901234567890
 auth_passphrase_hash_algorithm: sha
 auth_passphrase_hash_salt: 123456789012345678901234567890
 auth_passphrase_salt_algorithm: sha
 auth_passphrase_salt_salt: 123456789012345678901234567890
 auth_passphrase_hash_salt: 123456789012345678901234567890
 auth_passphrase_salt_salt: 123456789012345678901234567890
 auth_passphrase_hash_salt: 123456789012345678901234567890
 auth_passphrase_salt_hash: 123456789012345678901234567890
 state: present
 name: ansible_snmp_user
 location: US West 1
 contact: ansible@ansible.com
 description: Ansible SNMP User
 community: ansible
 contact_email: ansible@ansible.com
 contact_phone: 1234567890
 location_id: 123456789012345678901234567890
 location_name: US West 1
 location_description: Ansible SNMP User
 location_type: region
 location_ {{gen "value" dic=options_available}} {{#geneach "items" min_iterations=0 max_iterations=9 unique=unique}} {{#select "key" id=id}}aaa_local_user {{or}} acl_number {{or}} auth_key {{or}} auth_protocol {{or}} priv_key {{or}} priv_protocol {{or}} remote_engine_id {{or}} user_group {{or}} usm_user_name{{/select}} {{gen "value" dic=options_available}} {{/geneach}}


name: Create an SNMP user
  community.network.ce_snmp_user
  auth_protocol: MD5
  auth_passphrase: secret
  auth_user: admin
  auth_host: 10.10.10.10
  auth_port: 389
  auth_context: user
  auth_passphrase_hash: true
  auth_passphrase_algorithm: sha
  auth_passphrase_salt: 123456789012345678901234567890
  auth_passphrase_hash_algorithm: sha
  auth_passphrase_hash_salt: 123456789012345678901234567890
  auth_passphrase_salt_algorithm: sha
  auth_passphrase_salt_salt: 123456789012345678901234567890
  auth_passphrase_hash_salt: 123456789012345678901234567890
  auth_passphrase_salt_salt: 123456789012345678901234567890
  auth_passphrase_hash_salt: 123456789012345678901234567890
  auth_passphrase_salt_hash: 123456789012345678901234567890
  state: present
  name: ansible_snmp_user
  location: US West 1
  contact: ansible@ansible.com
  description: Ansible SNMP User
  community: ansible
  contact_email: ansible@ansible.com
  contact_phone: 1234567890
  location_id: 1234567890123456789012345678

In [52]:
starcoder_train_data = pd.read_parquet("./tldr_type_split/tldr_split_dataset_train_clean.parquet")
texttos = """aci_node_mgmt_epg"""

In [53]:
for i in range(len(starcoder_train_data)):
    if texttos in starcoder_train_data["o"]:
        print("present")
        print(i)

In [68]:
for i in range(240, 250):
    starcode_output_ft_out = starcode_output_ft["output"][i]
    gt = starcode_output_ft["o"][i]
    q = starcode_output_ft["q"][i]
    mod = starcode_output_ft["module"][i]
    print(starcode_output_ft_out)
    print("----")
    print(gt)
    print("----")
    print(q)
    print("----")
    print(mod)
    print("%%%%%%%%%%%%%%%%%%%%%")

In [64]:
print(starcode_output_ft_out)

---
# ----------------------------------------------------------------------------
#
#     ***     AUTO GENERATED CODE    ***    AUTO GENERATED CODE     ***
#
# ----------------------------------------------------------------------------
#
#     This file is automatically generated by Magic Modules and manual
#     changes will be clobbered when the file is regenerated.
#
#     Please read more about how to change this file at
#     https://www.github.com/GoogleCloudPlatform/magic-modules
#
# ----------------------------------------------------------------------------
# Pre-test setup
- name: create a config
  gcp_runtimeconfig_config:
    name: my-config
    description: My config
    project: "{{ gcp_project }}"
    auth_kind: "{{ gcp_cred_kind }}"
    service_account_file: "{{ gcp_cred_file }}"
    state: present
  register: config
- name: delete a variable
  gcp_runtimeconfig_variable:
    name: prod-variables/hostname
    config: my-config
    text: example.com
    project: "{{ gc

In [65]:
print(gt)

  gcp_runtimeconfig_variable:


In [66]:
print(q)

- name: delete a variable
